# Short-coding Project 2: Image enhancement
## Usage
Chose your image by changing the path variable. By default, the program will try to enhance the image using multiple transformations with the best possible values. However, it is possible to use those filters separetely, and also to change the parameters for some of them. 

## Specifications
### Noise reduction
As the title explains it well, this function reduces the noise present in the image. This is done by using a median filter (from the scikit-image module), by default with a convolution kernel of size 3x3. This size can be changed by putting in argument a 2-tuple (for example (10,10)). Note that this convolution is computed on the three separate RGB channels in the case of a color image, and on the only channel (obviously) for a grayscale image.

### Gamma correction
For a color image, if no gamma is specified, the gammaCorrection() method will apply a gamma function on the 'value' channel of the image (that would have then been converted from RGB to HSV). The $\gamma$-value is automatically calculated with the following formula: $\gamma = \dfrac{log(mid)}{log(mean)}$, where mid is the middle value possible (here 0.5 as we work with values from 0 to 1), and mean is the mean value of the 'value' channel of the image. This seems to be a good $\gamma$ based on the given image, and this formula is actually used by some image processing modules. If the image is a grayscale one, then the automatic $\gamma$ is calculated in a similar way but based on the intensity channel (we assume here a 256-value image, hence having 128 instead of 0.5 as the middle value).

If a gamma value is specified however, the gamma function will then be directly applied to the whole input image (and not on the 'value' channel only).


### Auto-level
Color image: the method applies an equalization of the input picture, to make its histogram as flat as possible. This is done by creating a lookup table based on the cumulative distribution function of the histogram.

Grayscale image: the method applies a contrast limited adaptive histogram equalization (CLAHE). Note that the implementation is very basic and thus requires a lot of computational power. The clip limit to be given is the absolute one, which will be normalized later. If none is specified, 1 is chosen. The number of bins of the histogram has as default value 128, and if the number of windows wanted in the x or y direction is not specified (default: 0), windows of 32 by 32 pixels will be created. These values were chosen arbitrarily (and tested on the 'camera.jpg' image provided) because calculating optimal values was quite complicated. Also, we assume that the images are 8-bit images (thus that the values range from 0 to 255). The code is based on Siladittya Manna's 'clahe' function.

### Saturation
The adaptSaturation() method will multiply the saturation channel of the image (transformed to HSV) by the 'factor' argument. If none (or 'auto') is given, an automatic factor is calculated to make the mean saturation value of the transformed image 0.5. This method only accepts color images.

### Enhance
Finally, the autoEnhance() method will apply all the methods stated above with automatic parameters. The order in which these filters are applied was studied, and the best order found is :
- First apply the automatic gamma correction, as it keeps quite well the 'fidelity' of the image
- Then adapt the saturation, because the gamma correction tends to give a 'bland' image (only for color images)
- Next, reduce the noise
- Finally, auto-level the image as it is the 'higher-level' processing (it takes into account the whole histogram of the image) and is thus best placed as a final step

## Output
If the code is kept unchanged (except for the path of the imput image of course), the output is given as a 2 by 2 table of images, having on the left top and bottom the original image, on the right top the image with only the gamma correction applied to it, and on the right bottom the auto enhance applied. This is done this way so it is easy to compare the original image to the changed ones, and the 'gamma-corrected-only' image is given as it preserves quite well the fidelity of the image and gives a more 'realistic' output (this is of course subjective), but unfortunately a bit too bland compared to the all-enhanced image.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage import color
from skimage.filters.rank import median
from skimage.io import imread,imshow,imsave
%matplotlib inline

path = 'etretat.jpg'

im = imread(path)


def noiseReduction(image, selemsize=(3,3)):
    selem = np.ones(selemsize)
    if len(np.shape(image))<3:
        newimage = median(image,selem)
    else:
        newimager = median(image[:,:,0],selem)
        newimageg = median(image[:,:,1],selem)
        newimageb = median(image[:,:,2],selem)
        newimage = np.dstack((newimager,newimageg,newimageb))
    return newimage 

def gammaCorrection(image, gamma='auto'):
    if gamma == 'auto' and len(np.shape(image))==3:
        imagehsv = color.rgb2hsv(image)
        h, s, v = imagehsv[:,:,0], imagehsv[:,:,1], imagehsv[:,:,2]
        gamma = np.log(0.5)/np.log(np.mean(v))
        newv = np.power(v,gamma)
        newimagehsv = np.dstack((h, s, newv))
        newimage = color.hsv2rgb(newimagehsv)
        return newimage
    elif gamma == 'auto' and len(np.shape(image))!=3:
        gamma = np.log(128)/np.log(np.mean(image))
    newimage = np.power(image/float(np.max(image)), gamma)
    newimage = (newimage*255).astype('uint8')
    return newimage

def autoLevel(image, clipLimit=1, nrBins=128, nrX=0, nrY=0):
    if len(np.shape(image))>2:
        hist,bins = np.histogram(image.flatten(),256)
        cdf = hist.cumsum()
        cdflut = ((cdf-np.min(cdf[np.nonzero(cdf)]))/(np.size(image)-np.min(cdf[np.nonzero(cdf)]))*255).astype('uint8')
        newimage = cdflut[image]
        return newimage
    else:
        h,w = np.shape(image)
        if nrX==0:
            xsz = 32
            ysz = 32
            nrX = int(np.ceil(h/xsz))
            nrY = int(np.ceil(w/ysz))

        else:
            xsz = round(h/nrX)
            ysz = round(w/nrY)

        nrPixels = xsz*ysz
        xsz2 = round(xsz/2)
        ysz2 = round(ysz/2)
        newimage = np.zeros(image.shape)

        #Normalizing clip limit
        clipLimit = max(1,clipLimit*xsz*ysz/nrBins)

        #LUT
        minVal = 0
        maxVal = 255

        binSz = np.floor(1+(maxVal-minVal)/float(nrBins))
        LUT = np.floor((np.arange(minVal,maxVal+1)-minVal)/float(binSz))

        bins = LUT[image]

        #Histogram
        hist = np.zeros((nrX,nrY,nrBins))
        for i in range(nrX):
            for j in range(nrY):
                binn = bins[i*xsz:(i+1)*xsz,j*ysz:(j+1)*ysz].astype(int)
                for i1 in range(xsz):
                    for j1 in range(ysz):
                        hist[i,j,binn[i1,j1]]+=1

        #Clip
        for i in range(nrX):
            for j in range(nrY):
                nrExcess = 0
                for nr in range(nrBins):
                    excess = hist[i,j,nr] - clipLimit
                    if excess>0:
                        nrExcess += excess

                binIncr = nrExcess/nrBins
                upper = clipLimit - binIncr
                for nr in range(nrBins):
                    if hist[i,j,nr] > clipLimit:
                        hist[i,j,nr] = clipLimit
                    else:
                        if hist[i,j,nr]>upper:
                            nrExcess += upper - hist[i,j,nr]
                            hist[i,j,nr] = clipLimit
                        else:
                            nrExcess -= binIncr
                            hist[i,j,nr] += binIncr

                if nrExcess > 0:
                    stepSz = max(1,np.floor(1+nrExcess/nrBins))
                    for nr in range(nrBins):
                        nrExcess -= stepSz
                        hist[i,j,nr] += stepSz
                        if nrExcess < 1:
                            break

        mapp = np.zeros((nrX,nrY,nrBins))
        scale = (maxVal - minVal)/float(nrPixels)
        for i in range(nrX):
            for j in range(nrY):
                summ = 0
                for nr in range(nrBins):
                    summ += hist[i,j,nr]
                    mapp[i,j,nr] = np.floor(min(minVal+summ*scale,maxVal))

        #Interpolation
        xI = 0
        for i in range(nrX+1):
            if i==0:
                subX = int(xsz/2)
                xU = 0
                xB = 0
            elif i==nrX:
                subX = int(xsz/2)
                xU = nrX-1
                xB = nrX-1
            else:
                subX = xsz
                xU = i-1
                xB = i

            yI = 0
            for j in range(nrY+1):
                if j==0:
                    subY = int(ysz/2)
                    yL = 0
                    yR = 0
                elif j==nrY:
                    subY = int(ysz/2)
                    yL = nrY-1
                    yR = nrY-1
                else:
                    subY = ysz
                    yL = j-1
                    yR = j
                UL = mapp[xU,yL,:]
                UR = mapp[xU,yR,:]
                BL = mapp[xB,yL,:]
                BR = mapp[xB,yR,:]
                subBin = bins[xI:xI+subX,yI:yI+subY]
                subImage = np.zeros(subBin.shape)
                num = subX*subY
                for i in range(subX):
                    inverseI = subX-i
                    for j in range(subY):
                        inverseJ = subY-j
                        val = subBin[i,j].astype(int)
                        subImage[i,j] = np.floor((inverseI*(inverseJ*UL[val] + j*UR[val])+ i*(inverseJ*BL[val] + j*BR[val]))/float(num))

                newimage[xI:xI+subX,yI:yI+subY] = subImage
                yI += subY
            xI += subX

        return newimage
    
def adaptSaturation(image,factor='auto'):
    if len(np.shape(image)) != 3:
        print('This method is only available for color images.')
        return 
    imagehsv = color.rgb2hsv(image)
    if factor == 'auto':
        meansat = np.mean(imagehsv[:,:,1])
        factor = 0.5/meansat
    imagehsv[:,:,1] *= factor    
    for i in range(np.size(imagehsv[:,:,1],0)):
        for j in range(np.size(imagehsv[:,:,1],1)):
            if imagehsv[i,j,1] > 1:
                imagehsv[i,j,1] = 1
    newimage = color.hsv2rgb(imagehsv)
    return newimage

def autoEnhance(image):
    if len(np.shape(image))<3:
        newimage = autoLevel(noiseReduction(gammaCorrection(image)))
        return newimage
    else:
        newimage = autoLevel(noiseReduction(adaptSaturation(gammaCorrection(image))))
        return newimage


newim1 = gammaCorrection(im)
newim2 = autoEnhance(im)

plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
plt.imshow(im,cmap='gray')
plt.title('Original')
plt.subplot(2,2,2)
plt.imshow(newim1,cmap='gray')
plt.title('Gamma correction')
plt.subplot(2,2,3)
plt.imshow(im,cmap='gray')
plt.title('Original')
plt.subplot(2,2,4)
plt.imshow(newim2,cmap='gray')
plt.title('All')
plt.show()      